In [1]:
%run ../Python_files/util.py
%run ../Python_files/util_ano_detec.py
%run ../Python_files/load_dicts.py

traffic_data_with_anomaly = zload('../temp_files/traffic_data_with_anomaly.pkz')
traffic_data_ref = zload('../temp_files/traffic_data_ref.pkz')

No dicts found; please check load_dicts...


In [2]:
def quantize(x, n, inf, sup):
    """
    x: the input data (scalar)
    n: quantization level
    inf: the lower bound of the input data x
    sup: the upper bound of the input data x
    -------------------
    Example:
    >>> quantize(4, 3, 2, 8)
    >>> 1
    >>> quantize(6.9, 3, 2, 8)
    >>> 2
    """
    if (x < inf):
        return 0
    elif (x >= sup):
        return (n-1)
    else:
        lev_length = (sup - inf) / float(n)
        return int(np.floor((x - inf) / lev_length))

In [3]:
tmc = '129-04138'
month = 1
day_list = [2, 3, 4, 5, 6]

traffic_data_ref_list = []
for hour in range(24):
    for minute in range(60):
        for day in day_list:
            key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
            traffic_data_ref_list.append(traffic_data_ref[key])

In [4]:
N = 8
inf = min(traffic_data_ref_list)
sup = max(traffic_data_ref_list)

In [5]:
traffic_data_ref_list_quantized = [quantize(s, N, inf, sup) for s in traffic_data_ref_list]

In [6]:
traffic_data_ref_list_quantized[0:10]

[5, 6, 5, 5, 5, 5, 6, 5, 5, 5]

In [11]:
mapping_dict = {}

for i in range(N):
    for j in range(N):
        mapping_dict[(i, j)] = i * N + j

In [12]:
mapping_dict

{(0, 0): 0,
 (0, 1): 1,
 (0, 2): 2,
 (0, 3): 3,
 (0, 4): 4,
 (0, 5): 5,
 (0, 6): 6,
 (0, 7): 7,
 (1, 0): 8,
 (1, 1): 9,
 (1, 2): 10,
 (1, 3): 11,
 (1, 4): 12,
 (1, 5): 13,
 (1, 6): 14,
 (1, 7): 15,
 (2, 0): 16,
 (2, 1): 17,
 (2, 2): 18,
 (2, 3): 19,
 (2, 4): 20,
 (2, 5): 21,
 (2, 6): 22,
 (2, 7): 23,
 (3, 0): 24,
 (3, 1): 25,
 (3, 2): 26,
 (3, 3): 27,
 (3, 4): 28,
 (3, 5): 29,
 (3, 6): 30,
 (3, 7): 31,
 (4, 0): 32,
 (4, 1): 33,
 (4, 2): 34,
 (4, 3): 35,
 (4, 4): 36,
 (4, 5): 37,
 (4, 6): 38,
 (4, 7): 39,
 (5, 0): 40,
 (5, 1): 41,
 (5, 2): 42,
 (5, 3): 43,
 (5, 4): 44,
 (5, 5): 45,
 (5, 6): 46,
 (5, 7): 47,
 (6, 0): 48,
 (6, 1): 49,
 (6, 2): 50,
 (6, 3): 51,
 (6, 4): 52,
 (6, 5): 53,
 (6, 6): 54,
 (6, 7): 55,
 (7, 0): 56,
 (7, 1): 57,
 (7, 2): 58,
 (7, 3): 59,
 (7, 4): 60,
 (7, 5): 61,
 (7, 6): 62,
 (7, 7): 63}

In [13]:
traffic_data_ref_list_quantized_trans = [mapping_dict[(traffic_data_ref_list_quantized[i], \
                                                       traffic_data_ref_list_quantized[i+1])] \
                                         for i in range(len(traffic_data_ref_list_quantized)-1)]

In [14]:
traffic_data_ref_list_quantized[0:10], traffic_data_ref_list_quantized_trans[0:10]

([5, 6, 5, 5, 5, 5, 6, 5, 5, 5], [46, 53, 45, 45, 45, 46, 53, 45, 45, 45])

In [15]:
tmc = '129-04138'
month = 1
day = 9

traffic_data_with_anomaly_list = []
for hour in range(24):
    for minute in range(60):
        key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
        traffic_data_with_anomaly_list.append(traffic_data_with_anomaly[key])

In [16]:
traffic_data_with_anomaly_list_quantized = [quantize(s, N, inf, sup) for s in traffic_data_with_anomaly_list]

In [17]:
traffic_data_with_anomaly_list_quantized_trans = \
[mapping_dict[(traffic_data_with_anomaly_list_quantized[i], \
               traffic_data_with_anomaly_list_quantized[i+1])] \
 for i in range(len(traffic_data_with_anomaly_list_quantized)-1)]

In [18]:
traffic_data_with_anomaly_list_quantized[0:10], traffic_data_with_anomaly_list_quantized_trans[0:10]

([7, 5, 5, 6, 6, 5, 5, 5, 5, 5], [61, 45, 46, 54, 53, 45, 45, 45, 45, 45])

In [19]:
len(traffic_data_with_anomaly_list_quantized_trans)

1439

In [20]:
mu_1 = mu_est(traffic_data_ref_list_quantized_trans, N)  # normal PL

In [21]:
mu_1

array([[  9.99999710e-09,   9.99999710e-09,   9.99999710e-09,
          9.99999710e-09,   9.99999710e-09,   9.72356990e-04,
          1.80580584e-03,   9.99999711e-09],
       [  9.99999711e-09,   9.99999711e-09,   9.99999711e-09,
          9.99999711e-09,   9.99999711e-09,   4.16724425e-04,
          1.38908142e-04,   9.99999711e-09],
       [  9.99999711e-09,   9.99999711e-09,   9.99999711e-09,
          9.99999711e-09,   9.99999711e-09,   9.99999711e-09,
          9.72356991e-04,   9.99999711e-09],
       [  9.99999711e-09,   9.99999711e-09,   9.99999711e-09,
          9.99999711e-09,   9.99999711e-09,   5.55632566e-04,
          2.22253027e-03,   4.16724425e-04],
       [  2.77816283e-04,   1.38908142e-04,   9.99999712e-09,
          9.99999712e-09,   9.99999712e-09,   3.05597912e-03,
          1.80580584e-03,   1.11126514e-03],
       [  1.11126514e-03,   1.38908142e-04,   9.99999714e-09,
          1.38908142e-03,   2.91707098e-03,   1.96971746e-01,
          1.25572968e-01,   5.0

In [22]:
N, _ = np.shape(mu_1)
assert(N == _)

mu_1 = mu_adjust(mu_1)  # normal PL

mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

zdump([mu1, mu_11, P1, G_11, H_11, U_11], '../temp_files/Traffic_ano_detec_PLs.pkz')

<string>:339: RuntimeWarning: covariance is not positive-semidefinite.


In [23]:
mu_11

array([[  1.56249956e-05,   9.99999720e-09,   9.99999720e-09,
          9.99999720e-09,   9.99999720e-09,   8.74999755e-04,
          1.85937448e-03,   9.99999721e-09],
       [  9.99999721e-09,   9.99999721e-09,   9.99999721e-09,
          9.99999721e-09,   9.99999721e-09,   3.90624891e-04,
          1.09374969e-04,   9.99999721e-09],
       [  9.99999721e-09,   9.99999721e-09,   9.99999721e-09,
          9.99999721e-09,   9.99999721e-09,   9.99999721e-09,
          9.84374725e-04,   9.99999721e-09],
       [  9.99999721e-09,   9.99999721e-09,   9.99999721e-09,
          9.99999721e-09,   9.99999721e-09,   6.56249817e-04,
          2.28124936e-03,   5.15624857e-04],
       [  2.65624926e-04,   1.09374970e-04,   9.99999722e-09,
          9.99999722e-09,   9.99999722e-09,   3.28124909e-03,
          2.06249943e-03,   1.34374963e-03],
       [  1.10937469e-03,   1.09374970e-04,   9.99999724e-09,
          1.37499962e-03,   3.20312412e-03,   1.96734321e-01,
          1.24187473e-01,   5.0

In [24]:
# n_range = [40, 70, 100]
# num_test_sample = 200
# beta_list = list(np.arange(0, 0.2, 0.02)[1:-1]) + list(np.arange(0.2, 1.01, 0.02)[:-1])

num_of_samp_per_win = 60
num_test_sample = 1380
beta = 0.001

In [25]:
eta_wc = {}
eta_Sanov = {}

n = num_of_samp_per_win

# Get thresholds for Hoeffding's test corresponding to sample length n    
key = str(n) + '_' + str(beta)
G_list = [G_11]
H_list = [H_11]
U_list = [U_11]
eta_1 = HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)
eta_2 =  - log(beta) / n
eta_wc[key] = eta_1
eta_Sanov[key] = eta_2
zdump([eta_wc, eta_Sanov], '../temp_files/testSample_threshold_traffic_ano_detec_%d.pkz'%N)

In [26]:
test_sample = []

n = num_of_samp_per_win

for idx in range(num_test_sample):
    test_sample.append(traffic_data_with_anomaly_list_quantized_trans[idx : (idx+60)])

In [27]:
KL = []
key = str(n) + '_' + str(beta)
for idx in range(num_test_sample):
    KL.append(KL_est(test_sample[idx], mu_11))
        
zdump(KL, '../temp_files/KL_%d_traffic_ano_detec.pkz'%N)

In [28]:
len(test_sample[1])

60

In [29]:
eta_wc, eta_Sanov

({'60_0.001': 0.77877554356831069}, {'60_0.001': 0.11512925464970228})

In [46]:
KL[975:1125]

[0.43524933775240449,
 0.44860272070688245,
 0.52603836465754239,
 0.60349239938464994,
 0.6809646128045872,
 0.7584557058037531,
 0.83596643025653794,
 0.91349758642802925,
 0.99105002554865917,
 1.0686246536758992,
 1.1462224362704221,
 1.2238444033906206,
 1.3014916555259888,
 1.3791653701334643,
 1.5124395679134306,
 1.5880542751294062,
 1.6641819825511985,
 1.7414957885768645,
 1.8188294419191564,
 1.8961839482553138,
 1.9735603917195486,
 2.050959943282002,
 2.128383870277931,
 2.2058335472818125,
 2.2833104685607455,
 2.3608162623905895,
 2.4383527075795906,
 2.5159217526210611,
 2.5935255379937616,
 2.6711664222521412,
 2.7488470127068876,
 2.6738460530133228,
 2.6738460530133228,
 2.6375910115697576,
 2.6396510227344261,
 2.6396510227344261,
 2.6396510227344261,
 2.6396510227344261,
 2.6752392200254156,
 2.7046184687837052,
 2.6576541206231039,
 2.6375910115697576,
 2.6319833266039803,
 2.6284572656355749,
 2.6284572656355749,
 2.6284572656355749,
 2.6284572656355749,
 2.62845

In [31]:
test_sample[0]

[61,
 45,
 46,
 54,
 53,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45,
 45]